In [1]:
import threading
import time
import random
import requests
import logging
from flask import Flask, request, jsonify

# Set up basic logging to monitor events and potential alerts
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configuration Parameters for vital sign thresholds
HEART_RATE_RANGE = (50, 120)         # Acceptable heart rate range (beats per minute)
BLOOD_PRESSURE_RANGE = (90, 140)       # Acceptable systolic blood pressure (mmHg)
OXYGEN_SAT_THRESHOLD = 95            # Minimum acceptable oxygen saturation (%)

In [2]:
# Create Flask app instance
app = Flask(__name__)

@app.route('/patient-data', methods=['POST'])
def patient_data():
    data = request.get_json()
    if not data:
        logging.error("No JSON data received.")
        return jsonify({"error": "Invalid data format"}), 400

    patient_id = data.get('patient_id', 'unknown')
    heart_rate = data.get('heart_rate')
    blood_pressure = data.get('blood_pressure')
    oxygen_saturation = data.get('oxygen_saturation')

    logging.info(f"Received data from Patient {patient_id}: HR={heart_rate}, BP={blood_pressure}, O2={oxygen_saturation}")

    # Simple anomaly detection based on configured thresholds
    anomalies = []
    if heart_rate is not None:
        if not (HEART_RATE_RANGE[0] <= heart_rate <= HEART_RATE_RANGE[1]):
            anomalies.append("Abnormal heart rate")
    else:
        anomalies.append("Missing heart rate reading")

    if blood_pressure is not None:
        if not (BLOOD_PRESSURE_RANGE[0] <= blood_pressure <= BLOOD_PRESSURE_RANGE[1]):
            anomalies.append("Abnormal blood pressure")
    else:
        anomalies.append("Missing blood pressure reading")

    if oxygen_saturation is not None:
        if oxygen_saturation < OXYGEN_SAT_THRESHOLD:
            anomalies.append("Low oxygen saturation")
    else:
        anomalies.append("Missing oxygen saturation reading")

    if anomalies:
        logging.warning(f"Anomalies for Patient {patient_id}: {anomalies}")

    return jsonify({"patient_id": patient_id, "anomalies": anomalies}), 200


In [3]:
def start_server():
    # Run the Flask app. Use host="0.0.0.0" to allow external access if needed.
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

# Start the Flask server in a separate thread so that the notebook remains interactive.
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True  # This ensures the thread will exit when the main process does
server_thread.start()

logging.info("Flask server started in background thread.")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [4]:
import time
import requests
import logging

# Wait a few seconds to ensure the Flask server has started
time.sleep(5)  # Adjust the sleep time if needed

def simulate_patient_sensor(patient_id):
    """Simulate patient vital readings and send data to the healthcare monitoring server."""
    while True:
        # Generate simulated sensor readings (some values may be intentionally abnormal)
        simulated_data = {
            "patient_id": patient_id,
            "heart_rate": round(random.uniform(40, 130), 2),
            "blood_pressure": round(random.uniform(80, 150), 2),
            "oxygen_saturation": round(random.uniform(90, 100), 2)
        }
        try:
            # Send the data to the Flask server endpoint
            response = requests.post("http://127.0.0.1:5000/patient-data", json=simulated_data)
            if response.status_code == 200:
                logging.info(f"Patient {patient_id}: Data processed successfully. Response: {response.json()}")
            else:
                logging.error(f"Patient {patient_id}: Failed to send data. Response: {response.text}")
        except Exception as e:
            logging.error(f"Patient {patient_id}: Exception occurred - {e}")
        # Wait before sending the next reading
        time.sleep(5)

# Start simulation for a single patient in a background thread.
sensor_thread = threading.Thread(target=simulate_patient_sensor, args=("P001",))
sensor_thread.daemon = True
sensor_thread.start()

logging.info("Patient sensor simulation started.")

INFO:werkzeug:127.0.0.1 - - [30/Mar/2025 18:38:37] "POST /patient-data HTTP/1.1" 200 -
